In [1]:
%load_ext autoreload
%autoreload 2

# IMPORT

In [16]:
import pandas as pd
from tqdm import tqdm

from src.process_tracking import process_data
from src.individual_pitch_control import extract_all_for_position_across_match, plot_half_pitch_individual_pitch_control, process_match

In [ ]:
fs = gcsfs.GCSFileSystem(token="google_default")

In [8]:
# Project_id GCP dans un .env

Some details regarding the run of this project:

- To complete this project, I had to save certain files and models as I go (to reduce RAM usage, avoid having to rerun everything each time, etc.). 

- My save and load functions are based on GCP, but they can easily be adapted to another cloud platform.

- However you can run everything in this notebook without saving files : in each section, I have included both a cell to run the code on a single file without saving the output, and a cell to run it on multiple files and save the output.

- All the steps run fast, except the train of the 2 Deep Learning models, so to make running easier, I have saved the two models in the repo in a lightweight format. 

# PROCESS TRACKING DATA

Before going into the core of the topic, we must prepare tracking data:
- Rescale coordinates to a 105x68 pitch
- Calculate velocities
- Retrieve meta data and merge with event data
- Normalize directions (always attacking left to right)

In [6]:
# RUN ON A SINGLE MATCH

df_processed = process_data(match_id=1886347, save_to_gcs=False)

[match 1886347] rows: 724617, players: 693106, ball: 31511


Merging match 1886347: 100%|██████████| 999/999 [00:04<00:00, 249.50it/s]


In [ ]:
# RUN ON ALL MATCHES AND SAVE OUTPUTS

match_ids = [1886347, 1899585, 1925299, 1953632, 1996435, 2006229, 2011166, 2013725, 2015213, 2017461]

for match_id in tqdm(match_ids, total=len(match_ids)):
    process_data(match_id=match_id, save_to_gcs=True)

# CALCULATE INDIVIDUAL PITCH CONTROL

As explained in the abstract, the objective of the project is to analyse and evaluate the space controled by a given player position in a given game situation.

This step allows to calculate what we call the "Individual Pitch Control", which is the Spearman's physics-based Pitch Control model definition, applied only to a single player : instead of summing the probabilities of each player of the teams, we only take into account the probability of a single player of a team and the sum of the probabilities of all the players of the opponent team. We do that to focus only on the space created by this player.
The output is a (32x50) map of the half-pitch.

We also calculate features that describe the game situation and that we will include in our clustering model (next step) : opponent defensive lines (calculated with an Agglomerative Clustering method) and contextual information about the player we focus on (distance from the ball, from nearest opponent, etc.)

In [13]:
player_position = "Center Forward"
game_situation = ('team_out_of_possession_phase_type','low_block')

In [17]:
# RUN ON A SINGLE MATCH

results = extract_all_for_position_across_match(
        df=df_processed,
        match_id=1886347,
        player_position=player_position,
        game_situation=game_situation,
        resolution=1
    )

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().